# Analysis datasets

The measurement plan for the Homepage experiment asks the following research questions:

* Does receiving the homepage treatment increase editor activation?
* Does receiving the homepage treatment increase editor retention?
* Does having access to the homepage treatment increase the average number of edits in the first two weeks after registration?
* Does having access to the homepage increase the proportion of constructive edits?

Additional questions from T218703:

* Having a confirmed email address
* Having created a user page (even if subsequently deleted)

For this analysis, I'd like to have a canonical dataset of users stored in the Data Lake such that it's easy to retrieve. This can then form the basis for gathering various datasets. One of the reasons for having a single source of truth is that we in this case have a complex set of criteria for selecting users.

For all the first four questions, I'd like to have data on both edits in the article namespace and other namespaces. That will enable us to learn whether the intervention affects either of those separately. This is particularly relevant because the Homepage has elements that make the first edit easier (e.g. starting a user page or asking your mentor a question). Both of those are outside the article namespace.

## Users in the dataset

We have multiple dates, wikis, and platforms for signing up that we need to deal with.

1. Between 2019-05-06T19:01:00Z (ref: [T217803#5168853](https://phabricator.wikimedia.org/T217803#5168853)) and 2019-06-26T16:12:32Z (ref: [T225676#5286530](https://phabricator.wikimedia.org/T225676#5286530)), the Homepage was only available on desktop, and only on Czech and Korean. We should therefore investigate whether users who signed up on mobile returned on desktop and viewed the homepage.
2. After 2019-06-26T16:12:32Z, the Homepage was available on both desktop and mobile. From that point on, use all registrations.
3. On 2019-07-01T18:29:22Z (ref: [T218237#5297442](https://phabricator.wikimedia.org/T218237#5297442)) the Homepage was launched on Vietnamese Wikipedia. From that point on, use all registrations on Vietnamese.
4. On 2019-07-24T23:39:39Z (ref: [T228120#5363655](https://phabricator.wikimedia.org/T228120#5363655)) the Homepage was launched on Arabic Wikipedia, so from that point on use all registrations there too.

In [2]:
import json
import datetime as dt

import pymysql

import numpy as np
import pandas as pd

from wmfdata import hive, mariadb
from growth import utils

You can find the source for `wmfdata` at https://github.com/neilpquinn/wmfdata


# Mobile registrations, desktop visits

If users who registered on the mobile site also don't visit the desktop Homepage before the mobile site was available, then we can put all mobile registrations into the control group and keep all desktop registrations in the treatment/control groups as defined by their user preferences.

To determine this, we'll use a single query to get all mobile, non-autocreated, non-API registrations from launch until two weeks before the mobile site was deployed, and see if any of these visited the Homepage within two weeks after registration. This two-week period covers the entire retention period used in our other analysis.

In [26]:
mob_visits_query = '''
WITH mob_regs AS (
    SELECT wiki, event.userid,
           unix_timestamp(dt, "yyyy-MM-dd'T'hh:mm:ss'Z'") AS reg_time
    FROM event_sanitized.serversideaccountcreation
    WHERE year = 2019
    AND month IN (5, 6)
    AND wiki IN ("cswiki", "kowiki")
    AND (event.isapi IS NULL
         OR event.isapi = false)
    AND event.isselfmade = true
    AND event.displaymobile = true
    AND dt > "2019-05-06T19:01:00Z"
    AND dt < "2019-06-12T00:00:00Z"
),
hp_visits AS (
    SELECT wiki, event.user_id,
           unix_timestamp(dt, "yyyy-MM-dd'T'hh:mm:ss'Z'") AS visit_time
    FROM event_sanitized.homepagevisit
    WHERE year = 2019
    AND month IN (5, 6)
    AND wiki IN ("cswiki", "kowiki")
)
SELECT mob_regs.wiki, mob_regs.userid, SUM(IF(hp_visits.user_id IS NOT NULL, 1, 0)) AS num_visits
FROM mob_regs
LEFT JOIN hp_visits
ON mob_regs.wiki = hp_visits.wiki
AND mob_regs.userid = hp_visits.user_id
WHERE (visit_time IS NULL
       OR visit_time - reg_time < 60*60*24*14) -- less than 14 days
GROUP BY mob_regs.wiki, mob_regs.userid
LIMIT 2500
'''

In [27]:
mob_visits = hive.run(mob_visits_query)

In [28]:
## How many users and how many visited?

mob_visits['visited'] = mob_visits['num_visits'] > 0

In [29]:
mob_visits.pivot_table(index = 'wiki', columns = 'visited', aggfunc = 'size')

visited,False,True
wiki,,
cswiki,511,13
kowiki,567,23


In [30]:
100 * pd.crosstab(mob_visits.wiki, mob_visits.visited, normalize = 'index').round(3)

visited,False,True
wiki,,
cswiki,97.5,2.5
kowiki,96.1,3.9


So we have 2.5% of mobile registrations on Czech, and 3.9% on Korean, visiting the Homepage within 14 days after registration. Because about 50% of these registrations are in the control group, it means that in the treatment group 5% in Czech, and 7.8% in Korean register on mobile and then later go on to visit the Homepage. That's definitely a large enough part of the treatment group that there is no point in trying to claim that mobile registrations don't carry over to the desktop site, it's clear that somewhere in the 5–10% range do, and it looks like it depends on the wiki. We could also speculate that these users are highly prolific ones, thereby making the control/treatment distinction difficult to make. So, we'll count *all registrations* from desktop deployment onwards as part of the experiment.

# Canonical user dataset

Gather a canonical dataset of user registrations (user ID, user registration timestamp), their treatment/control assignment, and whether they registered from desktop or mobile. This process follows the similar process for the Help Panel. We use `mediawiki_user_history` as the canonical source of registrations, and remove test accounts and bots at the same time. We then connect this dataset with `ServerSideAccountCreation` to get desktop/mobile data, filter out API creations, and verify autocreation status. Next, we use the MediaWiki table to get treatment/control assignment. Lastly, we use the `PrefUpdate` schema to filter out users who switched the Homepage option on/off.

In [10]:
## Configuration variables

wmf_snapshot = '2019-09'

wikis = ['cswiki', 'kowiki', 'viwiki', 'arwiki']

## Start and end timestamps of data gathering for each wiki, corresponding to the dates/times
## listed above.

cs_start_ts = dt.datetime(2019, 5, 6, 19, 1, 0)
cs_end_ts = dt.datetime(2019, 9, 15, 0, 0, 0)

ko_start_ts = cs_start_ts
ko_end_ts = cs_end_ts

vi_start_ts = dt.datetime(2019, 7, 1, 18, 29, 22)
vi_end_ts = cs_end_ts

ar_start_ts = dt.datetime(2019, 7, 24, 23, 39, 39)
ar_end_ts = cs_end_ts

## User IDs of known users to exclude (e.g. Stephane, Elena, and Marshall's accounts)
known_users = {
    'cswiki' : set([322106, 339583, 341191, 341611, 433381, 433382, 433511, 404765, 421667,
                      427625, 437386, 181724, 272273, 339583, 437386, 439783, 439792, 138342,
                      392634, 404765, 275298, 458487, 458049]),
    'kowiki' : set([384066, 539296, 539299, 539302, 539303, 539304, 539305, 539306, 539307,
                      539298, 416361, 416360, 413162, 495265, 518393, 518394, 518396, 530285,
                      531579, 531785, 536786, 536787, 542720, 542721, 542722, 543192, 543193,
                      544145, 544283, 470932, 38759, 555673]),
    'viwiki' : set(),
    'arwiki' : set()
}

## Filename of where the canonical dataset is stored.
tsv_output_file = "/home/nettrom/src/Growth-homepage-2019/datasets/canonical_users.tsv"

In [37]:
ko_known_users = set([384066, 539296, 539299, 539302, 539303, 539304, 539305, 539306, 539307,
                      539298, 416361, 416360, 413162, 495265, 518393, 518394, 518396, 530285,
                      531579, 531785, 536786, 536787, 542720, 542721, 542722, 543192, 543193,
                      544145, 544283, 470932, 38759, 555673])

username_patterns = ["MMiller", "Zilant", "Roan", "KHarlan", "MWang", "SBtest", "Rho2019"]
known_user_query = '''
SELECT user_id
FROM user
WHERE user_name LIKE "{name_pattern}%"
    '''

## Get known test accounts

In [49]:
## Grab the user IDs of known test accounts so they can be added to the exclusion list

def get_known_users(wiki):
    '''
    Get user IDs of known test accounts and return a set of them.
    '''
    
    username_patterns = ["MMiller", "Zilant", "Roan", "KHarlan", "MWang", "SBtest", "Rho2019"]

    known_user_query = '''
SELECT user_id
FROM user
WHERE user_name LIKE "{name_pattern}%"
    '''
    
    known_users = set()
    
    for u_pattern in username_patterns:
        new_known = mariadb.run(known_user_query.format(
            name_pattern = u_pattern), wiki)
        known_users = known_users | set(new_known['user_id'])

    return(known_users)
        
for wiki in wikis:
    known_users[wiki] = known_users[wiki] | get_known_users(wiki)

## Get all registrations

In [46]:
## Query to get user registrations on the given wikis for the given dates, ignoring
## known users.

user_registrations_query = '''
SELECT wiki_db, user_id, user_registration_timestamp
FROM wmf.mediawiki_user_history
WHERE snapshot = "{snapshot}"
AND caused_by_event_type = "create" -- account registration
AND created_by_self = true -- no auto-created accounts
AND SIZE(is_bot_by) = 0 -- no bots
AND SIZE(is_bot_by_historical) = 0 -- definitely no bots
AND (
    (wiki_db = "cswiki"
     AND user_registration_timestamp > "{cs_start_time}"
     AND user_registration_timestamp < "{cs_end_time}"
     AND user_id NOT IN ({cs_known_users}))
    OR
    (wiki_db = "kowiki"
     AND user_registration_timestamp > "{cs_start_time}"
     AND user_registration_timestamp < "{cs_end_time}"
     AND user_id NOT IN ({ko_known_users}))
    OR
    (wiki_db = "viwiki"
     AND user_registration_timestamp > "{vi_start_time}"
     AND user_registration_timestamp < "{cs_end_time}"
     AND user_id NOT IN ({vi_known_users}))
    OR
    (wiki_db = "arwiki"
     AND user_registration_timestamp > "{ar_start_time}"
     AND user_registration_timestamp < "{cs_end_time}"
     AND user_id NOT IN ({ar_known_users}))
)
'''

In [51]:
user_registrations = hive.run(
    user_registrations_query.format(
        snapshot = wmf_snapshot,
        cs_start_time = cs_start_ts.strftime(utils.hive_format),
        cs_end_time = cs_end_ts.strftime(utils.hive_format),
        vi_start_time = vi_start_ts.strftime(utils.hive_format),
        ar_start_time = ar_start_ts.strftime(utils.hive_format),
        cs_known_users = ', '.join([str(u) for u in known_users['cswiki']]),
        ko_known_users = ', '.join([str(u) for u in known_users['kowiki']]),
        vi_known_users = ', '.join([str(u) for u in known_users['viwiki']]),
        ar_known_users = ', '.join([str(u) for u in known_users['arwiki']]),
    )
)

## Get mobile/desktop and API data

This enables us to identify differences between mobile/desktop registrations. We also filter out API registrations as those are almost always mobile apps.

In [77]:
## Get all self-created registrations from ServerSideAccountCreation

ssac_reg_query = '''
SELECT wiki AS wiki_db, event.userid AS user_id,
       CAST(event.displaymobile AS INT) AS reg_on_mobile,
       event.isapi
FROM event_sanitized.serversideaccountcreation
WHERE year = 2019
AND month >= 5
AND wiki IN ("cswiki", "kowiki", "viwiki", "arwiki")
AND event.isselfmade = true
'''

In [78]:
ssac_reg_data = hive.run(ssac_reg_query)

In [79]:
## What's the number of registrations?

user_registrations.groupby('wiki_db').agg({'user_id' : 'size'})

,user_id
wiki_db,
arwiki,14771
cswiki,5415
kowiki,8040
viwiki,10136


In [80]:
ssac_reg_data.groupby('wiki_db').agg({'user_id' : 'size'})

,user_id
wiki_db,
arwiki,50907
cswiki,7330
kowiki,10978
viwiki,23102


In [99]:
## Join with SSAC data

all_users = user_registrations.merge(ssac_reg_data, how = 'left',
                                    on = ['wiki_db', 'user_id'])

In [100]:
## Did we lose anyone?

all_users.loc[all_users['reg_on_mobile'].isna()].groupby('wiki_db').agg({'user_id' : 'size'})

,user_id
wiki_db,
arwiki,7
cswiki,4
kowiki,1
viwiki,3


There's only a handful of users (on the grand scale of things) that we don't appear to have registration data for, so that means that we're good to go. Yay!

We remove users that have `isapi = True`, because those are most likely app registrations.

In [101]:
all_users = all_users.loc[~(all_users['isapi'] == True)]

In [102]:
## Drop the isapi columns

all_users = all_users.drop(columns = ['isapi'])

## Get treatment/control assignments

We grab all treatment assignments from the MediaWiki databases.

In [93]:
def get_panel_prefs(wiki, prop, users=None):
    '''
    Query and return a `pandas.DataFrame` with columns `wiki` and `user_id` of all users who have
    the Help Panel turned on in their preferences for that given wiki.
    
    :param wiki: database code of the wiki we're querying
    :type wiki: str
    
    :param prop: the user preference were use for determining treatment/control membership
    :type prop: str
    
    :param users: user IDs of the users we are interested in
    :type users: list
    '''
    
    panel_query = '''
    SELECT "{wiki}" AS wiki_db, up_user AS user_id,
           CAST(up_value AS UNSIGNED INTEGER) AS is_treatment
    FROM user_properties
    WHERE up_property = "{prop}"
    '''.format(wiki = wiki, prop = prop)
    
    if users is not None:
        panel_query += '''
        AND up_user IN ({})
        '''.format(','.join([str(uid) for uid in users]))
        
    return(mariadb.run(panel_query, wiki))

In [96]:
## Get treatment/control assignments from the MW databases

panel_prefs = pd.concat(
    [get_panel_prefs('cswiki', 'growthexperiments-homepage-enable',
                     all_users.loc[all_users.wiki_db == 'cswiki']['user_id']),
     get_panel_prefs('kowiki', 'growthexperiments-homepage-enable',
                     all_users.loc[all_users.wiki_db == 'kowiki']['user_id']),
     get_panel_prefs('viwiki', 'growthexperiments-homepage-enable',
                     all_users.loc[all_users.wiki_db == 'viwiki']['user_id']),
     get_panel_prefs('arwiki', 'growthexperiments-homepage-enable',
                     all_users.loc[all_users.wiki_db == 'arwiki']['user_id'])
    ])

In [103]:
all_users = all_users.merge(panel_prefs, on = ['wiki_db', 'user_id'], how = 'left').fillna(0)

## Users who turned the Homepage on/off in their preferences

Lastly, we identify all users who turned the Homepage on or off in their preferences, as that means they self-selected into or out of our group assignments. These users can therefore not be part of the analysis.

In [106]:
## Second, identify all users who either turned the Homepage on themselves, or at some point
## turned the preference off.

switch_query = '''
SELECT wiki, event.userid AS user_id, event.value
FROM event_sanitized.prefupdate
WHERE year = 2019
AND month >= 5
AND wiki IN ("cswiki", "kowiki", "viwiki", "arwiki")
AND event.property = "{prop}"
'''

In [107]:
switched_users = hive.run(
    switch_query.format(
        prop = 'growthexperiments-homepage-enable'
    )
)

How many users switched?

In [108]:
len(switched_users)

504

That's a surprisingly high number compared to other interventions we've run. I'm now curious to learn to what extent users turn this on or off, and how many users in the experiment who actually turned it on/off.

In [114]:
switched_users.groupby(['wiki', 'value']).agg({'user_id' : 'size'})

user_id
wiki   value         
arwiki false       24
       true       166
cswiki false       30
       true       130
kowiki false       11
       true        59
viwiki false        7
       true        77

Ok, so largely users are turning the Homepage *on*, not off. Given the number of users in our dataset, I don't think we're looking at a significant proportion turning it off.

In [115]:
## Left-join with switched users

all_users = all_users.merge(switched_users,
                            left_on = ['wiki_db', 'user_id'], right_on = ['wiki', 'user_id'],
                            how = 'left')

Now, for wiki and registration method, aggregate how many users turned it on or off, and how many didn't change it.

In [116]:
all_users.groupby(['wiki_db', 'reg_on_mobile', 'value']).agg({'user_id' : 'size'})

user_id
wiki_db reg_on_mobile value         
arwiki  0.0           false        6
                      true        33
        1.0           false        6
                      true        20
cswiki  0.0           false        4
                      true        14
kowiki  0.0           false        5
                      true         4
        1.0           false        1
                      true         1
viwiki  0.0           false        2
                      true        18
        1.0           false        1
                      true         6

In [118]:
len(all_users)

33301

In [119]:
len(all_users.loc[~all_users['value'].isna()])

121

In [120]:
round(100 * len(all_users.loc[~all_users['value'].isna()]) / len(all_users), 1)

0.4

So, at the time of writing (Oct 28, 2019), we've only had 121 out of 33,301 users in the experiment turn it on or off. Most of these turned it *on*, btw. That's 0.4% of users. We also have about 380 users who are not in the experiment turn the Homepage on. Cool!

In [121]:
all_users = all_users.loc[all_users['value'].isna()]

In [123]:
## Turn is_treatment and reg_on_mobile into ints

all_users['is_treatment'] = all_users['is_treatment'].astype(int)
all_users['reg_on_mobile'] = all_users['reg_on_mobile'].astype(int)

/home/nettrom/venv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/nettrom/venv/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [134]:
## Write the dataset out into a TSV so it can be imported into the Data Lake.
## NOTE: this file cannot contain a header line.

all_users.to_csv(tsv_output_file, sep = '\t',
                 columns = ['wiki_db', 'user_id', 'user_registration_timestamp',
                            'reg_on_mobile', 'is_treatment'],
                 header = False, index = False)

In [129]:
## Query to create the canonical user dataset table in Hive

create_table_query = '''
CREATE TABLE {table_name}
(wiki_db STRING COMMENT "wiki this user registered on",
 user_id BIGINT COMMENT "user_id of this user on the given wiki",
 user_registration_timestamp STRING COMMENT "UTC timestamp of user registration",
 reg_on_mobile INT COMMENT "whether the user registered on the mobile site",
 is_treatment INT COMMENT "whether the user is in the treatment or control group"
)
ROW FORMAT DELIMITED FIELDS TERMINATED BY "\t"
'''

In [130]:
canonical_user_table = 'nettrom_growth.homepage_exp_users'

hive.run(create_table_query.format(
    table_name = canonical_user_table
))

Importing the data was done manually through the command line in order to have a meaningful reference to the actual local file.

In [133]:
print(import_data_query.format(
    local_path = tsv_output_file,
    table_name = canonical_user_table
))


LOAD DATA LOCAL INPATH "/home/nettrom/src/Growth-homepage-2019/datasets/canonical_users.tsv"
OVERWRITE INTO TABLE nettrom_growth.homepage_exp_users



## Grabbing editing data

We have four questions mentioned in the introduction. As also mentioned in the introduction, we want to grab editing data to be able to answer these questions in such a way that we can analyze it separately for the article, user, and user talk namespaces separately from other namespaces. "Other" will likely mean "Wikipedia" or "Help" due to where the Help Desk is located. In other words, we can learn to what extent users are activated/retained by editing articles, user (creating their user page), talk (asking their mentor or responding to communication), and "other" (e.g. Help Desk).

* Does receiving the homepage treatment increase editor activation?
* Does receiving the homepage treatment increase editor retention?
* Does having access to the homepage treatment increase the average number of edits in the first two weeks after registration?
* Does having access to the homepage increase the proportion of constructive edits?

To accomplish this, we will join `mediawiki_history` with our Homepage user table to get edit data.

Note: we could also use edit tags to get mentor questions and Help Desk questions, specifically. There are benefits and drawbacks of both approaches. If we only look at those edits specifically, then further interaction (e.g. responding to your mentor's answer) will be different from those and counted differently. The way it is set up now, we use namespaces as proxies for specific actions and with that potentially count other things into it as well (e.g. a user might just edit their own user talk page without having asked their mentor).

I think we'll first start looking at the overall picture (activation and retention across all namespaces), and then take it from there if we find significant results.

In [154]:
edit_data_query = '''
WITH edits AS (
    SELECT wiki_db, event_user_id AS user_id,
    -- ns 0 edits on the first day
    SUM(IF(page_namespace = 0
        AND unix_timestamp(event_timestamp) - unix_timestamp(event_user_creation_timestamp) < 86400, 1, 0)) AS num_edits_ns0_24hrs,
    -- ns 0 edits on the first day that were reverted
    SUM(IF(page_namespace = 0 AND revision_is_identity_reverted = true AND revision_seconds_to_identity_revert < 60*60*48
        AND unix_timestamp(event_timestamp) - unix_timestamp(event_user_creation_timestamp) < 86400, 1, 0)) AS num_rev_edits_ns0_24hrs,
    -- ns 2 edits on the first day
    SUM(IF(page_namespace = 2
        AND unix_timestamp(event_timestamp) - unix_timestamp(event_user_creation_timestamp) < 86400, 1, 0)) AS num_edits_ns2_24hrs,
    -- ns 2 edits on the first day that were reverted
    SUM(IF(page_namespace = 2 AND revision_is_identity_reverted = true AND revision_seconds_to_identity_revert < 60*60*48
        AND unix_timestamp(event_timestamp) - unix_timestamp(event_user_creation_timestamp) < 86400, 1, 0)) AS num_rev_edits_ns2_24hrs,
    -- ns 3 edits on the first day,
    SUM(IF(page_namespace = 3
        AND unix_timestamp(event_timestamp) - unix_timestamp(event_user_creation_timestamp) < 86400, 1, 0)) AS num_edits_ns3_24hrs,
    -- ns 3 edits on the first day that were reverted
    SUM(IF(page_namespace = 3 AND revision_is_identity_reverted = true AND revision_seconds_to_identity_revert < 60*60*48
        AND unix_timestamp(event_timestamp) - unix_timestamp(event_user_creation_timestamp) < 86400, 1, 0)) AS num_rev_edits_ns3_24hrs,
    -- ns != (0, 2, 3) edits on the first day
    SUM(IF(page_namespace NOT IN (0, 2, 3)
        AND unix_timestamp(event_timestamp) - unix_timestamp(event_user_creation_timestamp) < 86400, 1, 0)) AS num_edits_other_24hrs,
    -- ns != (0, 2, 3) edits on the first day that were reverted
    SUM(IF(page_namespace NOT IN (0, 2, 3) AND revision_is_identity_reverted = true AND revision_seconds_to_identity_revert < 60*60*48
        AND unix_timestamp(event_timestamp) - unix_timestamp(event_user_creation_timestamp) < 86400, 1, 0)) AS num_rev_edits_other_24hrs,
    -- ns 0 edits on days 1–15
    SUM(IF(page_namespace = 0
        AND unix_timestamp(event_timestamp) - unix_timestamp(event_user_creation_timestamp) BETWEEN 86400 AND 15*86400, 1, 0)) AS num_edits_ns0_2w,
    -- ns 0 edits on days 1–15 that were reverted
    SUM(IF(page_namespace = 0 AND revision_is_identity_reverted = true AND revision_seconds_to_identity_revert < 60*60*48
        AND unix_timestamp(event_timestamp) - unix_timestamp(event_user_creation_timestamp) BETWEEN 86400 AND 15*86400, 1, 0)) AS num_rev_edits_ns0_2w,
    -- ns 2 edits on days 1–15
    SUM(IF(page_namespace = 2
        AND unix_timestamp(event_timestamp) - unix_timestamp(event_user_creation_timestamp) BETWEEN 86400 AND 15*86400, 1, 0)) AS num_edits_ns2_2w,
    -- ns 2 edits on days 1–15 that were reverted
    SUM(IF(page_namespace = 2 AND revision_is_identity_reverted = true AND revision_seconds_to_identity_revert < 60*60*48
        AND unix_timestamp(event_timestamp) - unix_timestamp(event_user_creation_timestamp) BETWEEN 86400 AND 15*86400, 1, 0)) AS num_rev_edits_ns2_2w,
    -- ns 3 edits on days 1–15
    SUM(IF(page_namespace = 3
        AND unix_timestamp(event_timestamp) - unix_timestamp(event_user_creation_timestamp) BETWEEN 86400 AND 15*86400, 1, 0)) AS num_edits_ns3_2w,
    -- ns 3 edits on days 1–15 that were reverted
    SUM(IF(page_namespace = 3 AND revision_is_identity_reverted = true AND revision_seconds_to_identity_revert < 60*60*48
        AND unix_timestamp(event_timestamp) - unix_timestamp(event_user_creation_timestamp) BETWEEN 86400 AND 15*86400, 1, 0)) AS num_rev_edits_ns3_2w,
    -- ns != (0, 2, 3) edits on days 1–15
    SUM(IF(page_namespace NOT IN(0, 2, 3)
        AND unix_timestamp(event_timestamp) - unix_timestamp(event_user_creation_timestamp) BETWEEN 86400 AND 15*86400, 1, 0)) AS num_edits_other_2w,
    -- ns != (0, 2, 3) edits on days 1–15 that were reverted
    SUM(IF(page_namespace NOT IN (0, 2, 3) AND revision_is_identity_reverted = true AND revision_seconds_to_identity_revert < 60*60*48
        AND unix_timestamp(event_timestamp) - unix_timestamp(event_user_creation_timestamp) BETWEEN 86400 AND 15*86400, 1, 0)) AS num_rev_edits_other_2w
    FROM wmf.mediawiki_history
    WHERE snapshot = "{snapshot}"
    AND event_entity = "revision"
    AND event_type = "create"
    AND wiki_db IN ("cswiki", "kowiki", "arwiki", "viwiki")
    AND event_timestamp > "2019-05-01"
    GROUP BY wiki_db, event_user_id
),
users AS (
    SELECT wiki_db, user_id, user_registration_timestamp, reg_on_mobile, is_treatment
    FROM {exp_user_table}
)
SELECT users.wiki_db, users.user_id, users.user_registration_timestamp, reg_on_mobile, is_treatment,
    COALESCE(num_edits_ns0_24hrs, 0) AS num_edits_ns0_24hrs,
    COALESCE(num_rev_edits_ns0_24hrs, 0) AS num_rev_edits_ns0_24hrs,
    COALESCE(num_edits_ns2_24hrs, 0) AS num_edits_ns2_24hrs,
    COALESCE(num_rev_edits_ns2_24hrs, 0) AS num_rev_edits_ns2_24hrs,
    COALESCE(num_edits_ns3_24hrs, 0) AS num_edits_ns3_24hrs,
    COALESCE(num_rev_edits_ns3_24hrs, 0) AS num_rev_edits_ns3_24hrs,
    COALESCE(num_edits_other_24hrs, 0) AS num_edits_other_24hrs,
    COALESCE(num_rev_edits_other_24hrs, 0) AS num_rev_edits_other_24hrs,
    COALESCE(num_edits_ns0_2w, 0) AS num_edits_ns0_2w,
    COALESCE(num_rev_edits_ns0_2w, 0) AS num_rev_edits_ns0_2w,
    COALESCE(num_edits_ns2_2w, 0) AS num_edits_ns2_2w,
    COALESCE(num_rev_edits_ns2_2w, 0) AS num_rev_edits_ns2_2w,
    COALESCE(num_edits_ns3_2w, 0) AS num_edits_ns3_2w,
    COALESCE(num_rev_edits_ns3_2w, 0) AS num_rev_edits_ns3_2w,
    COALESCE(num_edits_other_2w, 0) AS num_edits_other_2w,
    COALESCE(num_rev_edits_other_2w, 0) AS num_rev_edits_other_2w
FROM users
LEFT JOIN edits
ON users.wiki_db = edits.wiki_db
AND users.user_id = edits.user_id
'''

In [155]:
all_users_edit_data = hive.run(
    edit_data_query.format(
        snapshot = wmf_snapshot,
        exp_user_table = canonical_user_table
    )
)

In [156]:
len(all_users_edit_data)

33180

Write out the canonical edit dataset for importing into R.

In [158]:
all_users_edit_data.to_csv('datasets/all_users_edit_data.csv', index = False)

## Additional data

In order to answer the last two questions of analysis, we need to gather data about the following:

* Having a confirmed email address
* Having created a user page (even if subsequently deleted)

For both of these, I want to limit it to actions made within 14 days after registration. While some users might do these later, e.g. change their email address and then confirm it, we are primarily interested in effects on user behavior in the first few weeks on Wikipedia. Because historic email data is not stored anywhere, we won't have completely accurate data, but I suspect the effect of users later changing their address is minimal.

For creating a user page, I'll use `mediawiki_history` to gather that information and join it with the canonical dataset. We'll look for the creation of a page in the user namespace that matches `event_user_text_historical`, which should be the user's name at the time the page was created.

This analysis was done a few months after the initial one, so we start by loading the edit data back into memory.

In [15]:
all_users = pd.read_csv('datasets/all_users_edit_data.csv')

### Confirmed email address

We'll write a function to iterate over the wikis and grab confirmation data, only labelling users who confirmed it within 14 days.

In [19]:
def get_email_status(wiki, start_time, end_time):
    '''
    Query and return a `pandas.DataFrame` with columns `wiki`, `user_id`, and email
    verification status (1 if it was verified within 14 days after registration),
    for all users registered between `start_time` and `end_time`.
    
    :param wiki: database code of the wiki we're querying
    :type wiki: str

    :param start_time: timestamp of experiment deployment
    :type start_time: datetime.datetime
    
    :param end_time: timestamp of when to stop data gathering
    :type end_time: datetime.datetime
    '''

    email_query = '''
    SELECT "{wiki}" AS wiki_db, user_id,
        IF(user_email_authenticated IS NOT NULL
           AND TIMESTAMPDIFF(SECOND,
               TIMESTAMP(user_registration),
               TIMESTAMP(user_email_authenticated)) < 60*60*24*14, 1, 0) AS email_auth_14d
    FROM user
    WHERE user_registration >= "{start_ts}"
    AND user_registration < "{end_ts}"
    '''
        
    return(mariadb.run(email_query.format(
        wiki = wiki,
        start_ts = start_time.strftime(utils.mw_format),
        end_ts = end_time.strftime(utils.mw_format)
    ), wiki))

In [7]:
email_status = get_email_status('cswiki', cs_start_ts, cs_end_ts)

In [20]:
## Get treatment/control assignments from the MW databases

email_status = pd.concat(
    [get_email_status('cswiki', cs_start_ts, cs_end_ts),
     get_email_status('kowiki', ko_start_ts, ko_end_ts),
     get_email_status('viwiki', vi_start_ts, vi_end_ts),
     get_email_status('arwiki', ar_start_ts, ar_end_ts)
    ])

In [21]:
email_status.groupby(['wiki_db', 'email_auth_14d']).agg({'user_id': 'count'})

user_id
wiki_db email_auth_14d         
arwiki  0                 18497
        1                  4161
cswiki  0                 11526
        1                  2033
kowiki  0                 16712
        1                  1635
viwiki  0                 12420
        1                  2082

In [22]:
all_users = all_users.merge(email_status, on = ['wiki_db', 'user_id'], how = 'left').fillna(0)

In [24]:
## Fix float format when replacing NaNs
all_users['email_auth_14d'] = all_users['email_auth_14d'].astype(int)

### Created user page

For each of the wikis, we'll look for users who registered within the given timeframes, who created a page in the user space matching their user name at the time. We'll just grab all those edits and join it with all users, because that's easier than trying to filter.

In [27]:
userpage_query = '''
SELECT wiki_db, event_user_id, 1 AS create_userpage_14d
FROM wmf.mediawiki_history
WHERE snapshot = "{snapshot}"
AND event_entity = "revision"
AND event_type = "create"
AND wiki_db IN ("cswiki", "kowiki", "arwiki", "viwiki")
AND event_timestamp > "2019-05-01"
AND page_namespace = 2
AND unix_timestamp(event_timestamp) - unix_timestamp(event_user_creation_timestamp) < 86400*14
AND event_user_text_historical = page_title_historical
AND event_user_is_created_by_self = true
GROUP BY wiki_db, event_user_id
'''

In [28]:
userpage_data = hive.run(
    userpage_query.format(
        snapshot = wmf_snapshot
    ),
    spark_master = 'local[4]',
    spark_config = {'spark.driver.memory': '8g',
                    'spark.executor.memory' : '1g'})

In [31]:
all_users = all_users.merge(userpage_data, how = 'left',
                            left_on = ['wiki_db', 'user_id'],
                            right_on = ['wiki_db', 'event_user_id']).fillna(0)

In [32]:
## Fix float format when replacing NaNs
all_users['create_userpage_14d'] = all_users['create_userpage_14d'].astype(int)

In [34]:
## Write out this data to a CSV file, but only write the meaningful parts
all_users.to_csv('datasets/all_users_email_userpage_data.csv', index = False,
                 columns = ['wiki_db', 'user_id', 'user_registration_timestamp', 'reg_on_mobile',
                            'is_treatment', 'email_auth_14d', 'create_userpage_14d'])